In [1]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [2]:

# Step 1: Data cleaning
data = yf.download(tickers='BTC-usd', period='max', interval='1d')
data.dropna(inplace=True)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2023-01-10,17192.949219,17484.720703,17162.990234,17446.292969,17446.292969,15808338949
2023-01-11,17446.359375,17934.896484,17337.994141,17934.896484,17934.896484,18372283782
2023-01-12,18117.593750,19030.087891,17995.203125,18869.587891,18869.587891,34971338710


In [3]:

data.drop([ 'Close'], axis=1, inplace=True)
data

,Open,High,Low,Adj Close,Volume
Date,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100
...,...,...,...,...,...
2023-01-10,17192.949219,17484.720703,17162.990234,17446.292969,15808338949
2023-01-11,17446.359375,17934.896484,17337.994141,17934.896484,18372283782
2023-01-12,18117.593750,19030.087891,17995.203125,18869.587891,34971338710


In [4]:
# Step 3: Data normalization (cont.)
scaler = MinMaxScaler()
data_close = data[['Adj Close']]
data_close = scaler.fit_transform(data_close)
data_close

array([[0.00414359],
       [0.00365546],
       [0.00321557],
       ...,
       [0.27736813],
       [0.29280078],
       [0.3060737 ]])

In [12]:
# Step 4: Splitting the data
X = data[['Open', 'High', 'Low', 'Volume']]
X = scaler.fit_transform(X)
y = data_close
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [13]:
# Step 5: Reshaping the data
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
print("x", X.shape)
print('y',y.shape)
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('Y_test', y_test.shape) 
print('X_train.shape[1]', X_train.shape[1])
print('X_train.shape[2]', X_train.shape[2])

x (3042, 4)
y (3042, 1)
X_train (2433, 1, 4)
y_train (2433, 1)
X_test (609, 1, 4)
Y_test (609, 1)
X_train.shape[1] 1
X_train.shape[2] 4


In [16]:
# Step 6: Building the LSTM model (cont.)

model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True , activation= 'linear'))
model.add(Dropout(0.2))
model.add(LSTM(100, activation= 'linear'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Step 7: Training and evaluating the model
history = model.fit(X_train, y_train, epochs=250, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/250
77/77 [==============================] - 7s 16ms/step - loss: 0.0103 - val_loss: 0.0016
Epoch 2/250
77/77 [==============================] - 1s 9ms/step - loss: 4.3165e-04 - val_loss: 5.2493e-04
Epoch 3/250
77/77 [==============================] - 1s 8ms/step - loss: 3.3671e-04 - val_loss: 4.4813e-04
Epoch 4/250
77/77 [==============================] - 1s 9ms/step - loss: 3.1303e-04 - val_loss: 2.5205e-04
Epoch 5/250
77/77 [==============================] - 1s 11ms/step - loss: 2.7392e-04 - val_loss: 2.4023e-04
Epoch 6/250
77/77 [==============================] - 1s 12ms/step - loss: 2.8431e-04 - val_loss: 2.4504e-04
Epoch 7/250
77/77 [==============================] - 1s 10ms/step - loss: 2.6566e-04 - val_loss: 4.3698e-04
Epoch 8/250
77/77 [==============================] - 1s 10ms/step - loss: 2.4693e-04 - val_loss: 1.8271e-04
Epoch 9/250
77/77 [==============================] - 1s 10ms/step - loss: 2.1101e-04 - val_loss: 1.7396e-04
Epoch 10/250
77/77 [===================

In [17]:

# Step 8: Tuning the Hyperparameters
# You can use techniques such as grid search or random search to find the best values for the model's hyperparameters

# Step 9: Testing the model
y_pred = model.predict(X_test)
y_pred = y_pred.reshape(-1,1)
y_pred = np.nan_to_num(y_pred)
#y_pred = scaler.inverse_transform(y_pred)
#y_pred = scaler.inverse_transform(y_pred[:,[-1]])
#y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
y_test = np.nan_to_num(y_test)


#evaluat
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = 100 * np.mean(np.abs((y_test - y_pred) / y_test))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("MAPE: {:.2f}%".format(mape))
print("RMSE: {:.2f}".format(rmse))
print("MSE: {:.2f}".format(mse))
print("MAE:{:.2f}".format(mae))

20/20 [==============================] - 1s 4ms/step
MAPE: 1.75%
RMSE: 0.01
MSE: 0.00
MAE:0.01
